In [ ]:
from tensorflow.keras.layers import *
from keras import backend as K
import tensorflow as tf
from keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
def recall_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5),K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.clip(y_true, 0, 1))
    recall_ratio = true_positives / (possible_positives + K.epsilon())
    return recall_ratio

def precision_m(y_true, y_pred):

    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), 0.5), K.floatx())
    true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(y_pred)
    precision_ratio = true_positives / (predicted_positives + K.epsilon())
    return precision_ratio

def f1_m(y_true, y_pred):
    
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
img_width, img_height = 160, 160

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
batch_size = 128
epochs = 10

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.15)

training_set = train_datagen.flow_from_directory('../input/apparel-images-dataset',
                                                 target_size = (img_width, img_height),
                                                 batch_size = batch_size,
                                                 subset='training')

val_set = train_datagen.flow_from_directory('../input/apparel-images-dataset',
                                                 target_size = (img_width, img_height),
                                                 batch_size = batch_size,
                                                 subset='validation')

In [ ]:
fig, ax = plt.subplots(nrows=6, ncols=4,figsize = (15,20))

for X_batch, y_batch in training_set:
    i=0
    for row in ax:
        for col in row:
            col.imshow(X_batch[i])
            i+=1
    break
plt.show()    

In [ ]:
c = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",patience=5,
    verbose=1, mode="auto",
    restore_best_weights=True,
)

# ResNet50V2

In [ ]:
model = tf.keras.Sequential()

model.add( 
    tf.keras.applications.ResNet50V2(
        input_shape = input_shape,
        include_top = True,
        weights     = None,
        pooling     = 'max'
        )
)

model.add(Dense(512,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(24,activation='softmax'))

model.compile(
    optimizer = 'adam', 
    loss = 'categorical_crossentropy',               
    metrics=['accuracy']
)

model.summary()

In [ ]:
history = model.fit(training_set,
                    validation_data = val_set,
                    epochs=25,
                    callbacks = [c],
                    verbose = 1)

In [ ]:
print(f"Max F1-score in validation dataset = {max(history.history['val_accuracy'])}")

In [ ]:
# img-p = '/kaggle/input/apparel-images-dataset/black_shirt/010dd4d14ca70e8f8713437bf4849ebe21225f8e.jpg'
# from keras.preprocessing.image import img_to_array, load_img
# import numpy as np

# # Load the image
# img_path = '/kaggle/input/apparel-images-dataset/black_shirt/010dd4d14ca70e8f8713437bf4849ebe21225f8e.jpg'
# img = load_img(img_path, target_size=(img_width, img_height))

# # Convert the image to an array
# img_array = img_to_array(img)

# # Expand the dimensions to match the input shape expected by the model
# img_array = np.expand_dims(img_array, axis=0)

# # Preprocess the image using the same preprocessing function used in training
# preprocessed_img = train_datagen.standardize(img_array)

# # Make predictions using the model
# predictions = model.predict(preprocessed_img)
# print(predictions)
# model.predict('/kaggle/input/apparel-images-dataset/black_shirt/010dd4d14ca70e8f8713437bf4849ebe21225f8e.jpg')

In [ ]:
from keras.preprocessing import image
import numpy as np

# # Load the trained model
# model_path = '/kaggle/working/nsut-model'  # Path to your trained model file
# model = tf.keras.models.load_model(model_path)

# Function to preprocess the user-uploaded photo
def preprocess_user_photo(img_path, target_size=(img_width, img_height)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale pixel values to [0, 1] like training data
    return img_array

# Path to the user-uploaded photol

# Preprocess the user-uploaded photo
preprocessed_user_photo = preprocess_user_photo('/kaggle/input/apparel-images-dataset/brown_pants/00bc7186fc623c91bfa12607fd7e595ddd320617.jpg')

# Make predictions using the model
predictions = model.predict(preprocessed_user_photo)

# Get the predicted class label
predicted_class_index = np.argmax(predictions)
dict1=['dress','pants','shirt','shoes','shorts','dress','pants','shirt','shoes','shorts','pants','shoes','shorts','pants','shoes','shorts','dress','pants','shoes','dress','pants','shoes','shorts']
print(dict1[predicted_class_index])

In [ ]:
model.save("nsut-model")

In [ ]:
model.save("/kaggle/working/nsut-model")

In [ ]:
# Example of saving a TensorFlow model as an HDF5 file
model.save('/kaggle/working/model-ai.h5')

In [ ]:
rahul = tf.keras.models.load_model('/kaggle/working/model-ai.h5')

In [ ]:
from keras.preprocessing import image
import numpy as np

# # Load the trained model
# model_path = '/kaggle/working/nsut-model'  # Path to your trained model file
# model = tf.keras.models.load_model(model_path)

# Function to preprocess the user-uploaded photo
def preprocess_user_photo(img_path, target_size=(img_width, img_height)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale pixel values to [0, 1] like training data
    return img_array

# Path to the user-uploaded photo

# Preprocess the user-uploaded photo
preprocessed_user_photo = preprocess_user_photo('/kaggle/input/apparel-images-dataset/brown_pants/00bc7186fc623c91bfa12607fd7e595ddd320617.jpg')

# Make predictions using the model
predictions = rahul.predict(preprocessed_user_photo)

# Get the predicted class label
predicted_class_index = np.argmax(predictions)
dict1=['dress','pants','shirt','shoes','shorts','dress','pants','shirt','shoes','shorts','pants','shoes','shorts','pants','shoes','shorts','dress','pants','shoes','dress','pants','shoes','shorts']
print(dict1[predicted_class_index])

# Stacking

In [ ]:
def models(input_size = input_shape):

    inputs = tf.keras.layers.Input(input_size)
    
    base_model1 =     tf.keras.applications.ResNet50V2(
        include_top = False,
        weights     = None,
        pooling     = 'max'
        )(inputs)

    base_model2 =     tf.keras.applications.Xception(
        include_top = False,
        weights     = None,
        pooling     = 'max'
        )(inputs)
    
    model = tf.keras.layers.Concatenate()([base_model1,base_model2])
    
    model = tf.keras.layers.Dense(512, activation='relu')(model)
    model = tf.keras.layers.Dense(64, activation='relu')(model)
    model = tf.keras.layers.Dense(24, activation='softmax')(model)

    M = tf.keras.Model(inputs=inputs, outputs=model)
    M.compile(
        optimizer = 'adam', 
        loss = 'categorical_crossentropy',               
        metrics=['accuracy',f1_m]
    )
    
    return M

model = models()
model.summary()

In [ ]:
history = model.fit(training_set,
                    validation_data = val_set,
                    epochs=epochs,
                    callbacks = [c],
                    verbose = 1)

In [ ]:
print(f"Max F1-score in validation dataset = {max(history.history['val_f1_m'])}")